# Epymorph dev log: October, 13, 2023

Author: Alexander Nez

as of git hash: 420cc7921348d0de663cceb1fb18904c21941127


In [1]:
from epymorph.data import geo_library, ipm_library, mm_library


In [2]:
import pandas as pd
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from PIL import Image
import os
import requests
import time
import ast
from matplotlib.patches import ConnectionPatch
import random
from IPython.display import Image
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import matplotlib.patches as mpatches
import random
from shapely.geometry import mapping
import matplotlib.colors as mcolors


In [3]:
dir = os.path.expanduser('~/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/Movement_model')
Maricopa_schools_csv = os.path.expanduser('~/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/Movement_model/School_movement/Maricopa_schools.csv')
Maricopa_school = pd.read_csv(Maricopa_schools_csv)

In [4]:
roads_gdf = gpd.read_file(f"{dir}/tl_rd22_04013_roads/tl_rd22_04013_roads.shp")

maricopa_roads = roads_gdf[roads_gdf['MTFCC'].isin(['S1100', 'S1200'])]

maricopa_main_roads = roads_gdf[roads_gdf['MTFCC'] == 'S1100']
maricopa_secondary_roads = roads_gdf[roads_gdf['MTFCC'] == 'S1200']

In [5]:
from pygris import tracts 
from pygris import block_groups 
from pygris import primary_secondary_roads
from pygris import school_districts
from pygris.data import get_lodes
import pandas as pd

az_od = get_lodes(state="AZ", year=2020, lodes_type="od", cache=True)

maricopa_od = az_od[az_od['w_geocode'].astype(str).str.startswith('04013')]
maricopa_od = az_od[az_od['h_geocode'].astype(str).str.startswith('04013')]

districts = school_districts(state="AZ", year=2019)
roads = primary_secondary_roads(state="AZ", year=2019)
maricopa_tracts = tracts(state="AZ", county="Maricopa", year=2019, cache=False)
maricopa_block_groups = block_groups(state="AZ", county="Maricopa", year=2019, cache=False)

Using FIPS code '04' for input 'AZ'
Using FIPS code '04' for input 'AZ'
Using FIPS code '04' for input 'AZ'
Using FIPS code '013' for input 'Maricopa'
Using FIPS code '04' for input 'AZ'
Using FIPS code '013' for input 'Maricopa'


In [ ]:
state_fips = '08'
county_fips = '029'
districts = school_districts(state="AZ", year=2019)


In [6]:
with np.load(f"{dir}/maricopa_cbg_2019_geo.npz", allow_pickle=True) as npz_data:
    maricopa_geo = dict(npz_data)
    
census_df = pd.DataFrame({k: list(v) for k, v in maricopa_geo.items()})
census_df.head()

,label,tract_geoid,centroid,population,population_MOE,population_by_age,population_by_age_MOE,population_by_school,population_by_school_MOE,tract_preschool_enrollment,...,tract_kindergarten_enrollment_MOE,tract_elementary_enrollment_MOE,tract_middle_school_enrollment_MOE,tract_high_school_enrollment_MOE,median_age,median_income,average_household_size,pop_density_km2,tract_median_income,tract_gini_index
0,040130101011,04013010101,"[-111.77075643330394, 33.76924660709943]",1791,388.0,"[116, 33, 100, 109, 1155, 278]","[109.60383204979651, 57.271284253105414, 75.95...","[384, 140, 0, 139]","[186.0, 118.6001686339442, 20.784609690826528,...",546,...,104.0,17.0,71.0,82.0,50.5,99219,2.33,99.319999,99489,0.4423
1,040130101012,04013010101,"[-111.72000671148204, 33.7410933260103]",2007,472.0,"[34, 72, 0, 56, 883, 962]","[55.31726674375732, 69.35416353759881, 16.9705...","[162, 106, 0, 16]","[146.0, 118.41030360572512, 20.784609690826528...",546,...,104.0,17.0,71.0,82.0,63.9,127614,2.15,68.782398,99489,0.4423
2,040130101013,04013010101,"[-111.66697569766907, 33.72404113568911]",1264,245.0,"[0, 0, 0, 0, 271, 993]","[16.97056274847714, 16.97056274847714, 16.9705...","[0, 0, 0, 0]","[12.0, 29.393876913398138, 20.784609690826528,...",546,...,104.0,17.0,71.0,82.0,70.8,80742,1.83,183.890869,99489,0.4423
3,040130101021,04013010102,"[-111.49136471795472, 33.71546544071483]",1139,438.0,"[0, 10, 10, 38, 775, 306]","[16.97056274847714, 24.186773244895647, 32.310...","[91, 10, 0, 48]","[71.0, 34.07345007480164, 20.784609690826528, ...",384,...,17.0,17.0,57.0,47.0,58.9,105987,2.04,0.480805,134865,0.5439
4,040130101022,04013010102,"[-111.8505100649063, 33.82609833466325]",1487,410.0,"[0, 40, 39, 31, 784, 593]","[16.97056274847714, 51.419840528729765, 56.302...","[131, 40, 28, 33]","[106.0, 61.5223536610881, 55.65069631190611, 4...",384,...,17.0,17.0,57.0,47.0,62.8,128523,2.20,125.420289,134865,0.5439


# Functions


In [7]:
def random_color():
    r, g, b = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    return (r / 255.0, g / 255.0, b / 255.0, 1.0)

def haversine(centroid_1_lon: float, centroid_1_lat: float,
              centroid_2_lon: float, centroid_2_lat: float) -> np.double:
    R = 3959.87433
    dLat = np.radians(centroid_1_lat - centroid_2_lat)
    dLon = np.radians(centroid_1_lon - centroid_2_lon)
    lat1 = np.radians(centroid_1_lat)
    lat2 = np.radians(centroid_2_lat)

    a = np.sin(dLat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dLon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

def classify_school_type(low_grade, high_grade):
    if low_grade in ['PK'] and high_grade in ['PK']:
        return 'Preschool'
    if low_grade in ['KG'] and high_grade in ['KG']:
        return 'kindergarten School'
    if low_grade in ['PK', 'KG', '01', '02', '03', '04', '05'] and high_grade in ['PK', 'KG', '01', '02', '03', '04', '05']:
        return 'Elementary School'
    elif low_grade in ['06', '07', '08'] and high_grade in ['06', '07', '08']:
        return 'Middle School'
    elif low_grade in ['09', '10', '11', '12'] and high_grade in ['09', '10', '11', '12']:
        return 'High School'
    elif low_grade in ['PK', 'KG', '01', '02', '03', '04', '05'] and high_grade in ['06', '07', '08']:
        return 'Elementary/Middle School'
    elif low_grade in ['06', '07', '08'] and high_grade in ['09', '10', '11', '12']:
        return 'Middle/High School'
    else:
        return 'Elementary/Middle/High School'
    
def adjust_lightness(color, amount=0.5):
    """
    Adjust the lightness of the given color.
    `amount` > 1 will lighten the color, < 1 will darken it.
    color: RGBA tuple where each component is in the range 0 to 1
    """
    r, g, b, a = color
    r = min(r * amount, 1.0)
    g = min(g * amount, 1.0)
    b = min(b * amount, 1.0)
    return (r, g, b, a)

def distribute_students_proportionally(total_students, age_group_population):
    total_population = sum(age_group_population)
    
    if total_population == 0:
        return [0] * len(age_group_population)
    
    if total_population < total_students:
        raise ValueError("Not enough individuals to distribute students.")

    proportion = [x / total_population for x in age_group_population]
    distributed_students = [int(p * total_students) for p in proportion]

    remaining_students = total_students - sum(distributed_students)
    for i in range(len(age_group_population)):
        if age_group_population[i] > 0 and remaining_students > 0:
            distributed_students[i] += 1
            remaining_students -= 1

    return distributed_students

In [8]:
shp_path = "/Users/alexandernez/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/Movement_model/School_movement/School_Districts/School_Districts.shp"

school_districts = gpd.read_file(shp_path)
random_colors = [random_color() for _ in range(len(school_districts))]

school_districts['color'] = random_colors

school_districts['Centroid'] = school_districts['geometry'].representative_point()
school_districts['coords'] = school_districts['Centroid'].apply(lambda point: (point.x, point.y))

school_districts = school_districts.to_crs(epsg=4326)  

# Maricopa Public Schools


In [4]:
file_name = "shapefiles/EDGE_GEOCODE_PUBLICSCH_1920/EDGE_GEOCODE_PUBLICSCH_1920.xlsx"

full_file_path = os.path.join(dir, file_name)

columns_to_read = [
    'NCESSCH', 'CNTY', 'LAT', 'LON', 'SCHOOLYEAR'
]

NCES_Public_schools = pd.read_excel(full_file_path, dtype = {"CNTY": object}, usecols=columns_to_read)

az_public_schools = NCES_Public_schools[NCES_Public_schools['CNTY'] == '04017']
az_public_schools.drop(columns=['CNTY'], inplace=True)

az_public_schools.head()

/var/folders/_v/4q2_l88n1vndy51_ksbfylnc0000gn/T/ipykernel_90422/2035469379.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  az_public_schools.drop(columns=['CNTY'], inplace=True)


,NCESSCH,LAT,LON,SCHOOLYEAR
2092,40002301352,36.1110,-110.2155,2019-2020
2093,40002301355,36.1120,-110.2165,2019-2020
2094,40002301524,36.1148,-110.2161,2019-2020
2095,40002601389,34.4264,-110.5970,2019-2020
2096,40002601459,34.4275,-110.5967,2019-2020


In [10]:
file_name = "School_movement/NCES_public_schools.xltx"

full_file_path = os.path.join(dir, file_name)

columns_to_read = [
    'NCES School ID', 'Low Grade*',
    'High Grade*', 'School Name', 'District', 
    'Locale Code*', 'Locale*', 'Title I School*', 'Title 1 School Wide*', 'Students*',
    'Teachers*', 'Student Teacher Ratio*', 'Type', 'Status'
]

NCES_public_schools = pd.read_excel(full_file_path, skiprows=14, usecols=columns_to_read)
    
NCES_public_schools['school type'] = NCES_public_schools.apply(lambda row: classify_school_type(row['Low Grade*'], row['High Grade*']), axis=1)
NCES_public_schools['Students*'] = pd.to_numeric(NCES_public_schools['Students*'], errors='coerce')

NCES_public_schools.head()

,NCES School ID,Low Grade*,High Grade*,School Name,District,Locale Code*,Locale*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Type,Status,school type
0,40010601892,09,12,AAEC - Paradise Valley,Arizona Agribusiness & Equine Center Inc. (4331),11,City: Large,Yes,No,322.0,–,†,Regular,Open,High School
1,40041701588,09,12,AAEC - SMCC Campus,Arizona Agribusiness & Equine Center Inc. (85816),11,City: Large,Yes,Yes,499.0,–,†,Regular,Open,High School
2,40096903751,09,12,AAEC Online,Arizona Agribusiness & Equine Center INC. (134...,11,City: Large,No,†,42.0,–,†,Regular,Open,High School
3,40906000652,PK,08,Abraham Lincoln Traditional School,Washington Elementary School District (4260),11,City: Large,No,†,530.0,34.30000,15.4500000,Regular,Open,Elementary/Middle School
4,40906000915,KG,06,Acacia Elementary School,Washington Elementary School District (4260),11,City: Large,Yes,Yes,763.0,44.50000,17.1500000,Regular,Open,Elementary/Middle School


In [11]:
NCES_public_schools_df = pd.merge(NCES_public_schools, az_public_schools, left_on='NCES School ID', right_on='NCESSCH')

NCES_public_schools_df.drop(columns=['NCESSCH'], inplace=True)

NCES_public_schools_df.head()

,NCES School ID,Low Grade*,High Grade*,School Name,District,Locale Code*,Locale*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Type,Status,school type,LAT,LON,SCHOOLYEAR
0,40010601892,09,12,AAEC - Paradise Valley,Arizona Agribusiness & Equine Center Inc. (4331),11,City: Large,Yes,No,322.0,–,†,Regular,Open,High School,33.654305,-112.001380,2019-2020
1,40041701588,09,12,AAEC - SMCC Campus,Arizona Agribusiness & Equine Center Inc. (85816),11,City: Large,Yes,Yes,499.0,–,†,Regular,Open,High School,33.381603,-112.033968,2019-2020
2,40906000652,PK,08,Abraham Lincoln Traditional School,Washington Elementary School District (4260),11,City: Large,No,†,530.0,34.30000,15.4500000,Regular,Open,Elementary/Middle School,33.581300,-112.144000,2019-2020
3,40906000915,KG,06,Acacia Elementary School,Washington Elementary School District (4260),11,City: Large,Yes,Yes,763.0,44.50000,17.1500000,Regular,Open,Elementary/Middle School,33.616200,-112.123800,2019-2020
4,40089103676,KG,08,Academy of Math and Science Avondale,Academy of Mathematics and Science South Inc. ...,11,City: Large,Yes,Yes,1079.0,–,†,Regular,Open,Elementary/Middle School,33.434499,-112.286551,2019-2020


# Maricopa Private Schools


In [12]:
file_name = "shapefiles/EDGE_GEOCODE_PRIVATESCH_1920/EDGE_GEOCODE_PRIVATESCH_1920.xlsx"

full_file_path = os.path.join(dir, file_name)
columns_to_read = [
    'PPIN', 'CNTY', 'LOCALE', 'LAT', 'LON', 'SCHOOLYEAR'
]

NCES_private_schools = pd.read_excel(full_file_path, dtype = {"CNTY": object},  usecols=columns_to_read)

az_private_schools = NCES_private_schools[NCES_private_schools['CNTY'] == '04013']
az_private_schools.head()

,PPIN,CNTY,LOCALE,LAT,LON,SCHOOLYEAR
82,00031163,04013,11,33.451923,-112.104221,2019-2020
83,00031185,04013,11,33.418688,-111.833088,2019-2020
84,00031243,04013,11,33.492160,-111.916111,2019-2020
86,00031265,04013,11,33.505110,-112.067531,2019-2020
87,00031276,04013,11,33.485542,-112.170626,2019-2020


In [13]:
file_name = "School_movement/NCES_dataset.xltx"

full_file_path = os.path.join(dir, file_name)

columns_to_read = [
    'PSS_SCHOOL_ID', 'PSS_INST', 'PSS_ADDRESS', 'PSS_CITY', 'PSS_COUNTY_FIPS',
    'PSS_COUNTY_NAME', 'PSS_COUNTY_NO', 'PSS_FIPS', 'LoGrade', 'HiGrade',
    'PSS_ENROLL_UG', 'PSS_ENROLL_PK', 'PSS_ENROLL_K', 'PSS_ENROLL_1', 'PSS_ENROLL_2',
    'PSS_ENROLL_3', 'PSS_ENROLL_4', 'PSS_ENROLL_5', 'PSS_ENROLL_6', 'PSS_ENROLL_7',
    'PSS_ENROLL_8', 'PSS_ENROLL_9', 'PSS_ENROLL_10', 'PSS_ENROLL_11',
    'PSS_ENROLL_12', 'PSS_ENROLL_T', 'PSS_ENROLL_TK12',
]

NCES_private_schools = pd.read_excel(full_file_path, skiprows=4, usecols=columns_to_read)

columns_to_replace_nan = [
    'PSS_ENROLL_UG', 'PSS_ENROLL_PK', 'PSS_ENROLL_K', 'PSS_ENROLL_1', 'PSS_ENROLL_2',
    'PSS_ENROLL_3', 'PSS_ENROLL_4', 'PSS_ENROLL_5', 'PSS_ENROLL_6', 'PSS_ENROLL_7',
    'PSS_ENROLL_8', 'PSS_ENROLL_9', 'PSS_ENROLL_10', 'PSS_ENROLL_11', 'PSS_ENROLL_12'
]

NCES_private_schools[columns_to_replace_nan] = NCES_private_schools[columns_to_replace_nan].fillna(0)

NCES_private_schools['PSS_ENROLL_elementary'] = NCES_private_schools[['PSS_ENROLL_K', 'PSS_ENROLL_1', 'PSS_ENROLL_2', 'PSS_ENROLL_3', 'PSS_ENROLL_4', 'PSS_ENROLL_5']].sum(axis=1)
NCES_private_schools['PSS_ENROLL_middle_school'] = NCES_private_schools[['PSS_ENROLL_6', 'PSS_ENROLL_7', 'PSS_ENROLL_8']].sum(axis=1)
NCES_private_schools['PSS_ENROLL_high_school'] = NCES_private_schools[['PSS_ENROLL_9', 'PSS_ENROLL_10', 'PSS_ENROLL_11', 'PSS_ENROLL_12']].sum(axis=1)

columns_to_drop = [
    'PSS_ENROLL_K', 'PSS_ENROLL_1', 'PSS_ENROLL_2', 'PSS_ENROLL_3', 'PSS_ENROLL_4', 'PSS_ENROLL_5',
    'PSS_ENROLL_6', 'PSS_ENROLL_7', 'PSS_ENROLL_8', 'PSS_ENROLL_9', 'PSS_ENROLL_10', 'PSS_ENROLL_11', 'PSS_ENROLL_12'
]

NCES_private_schools.drop(columns=columns_to_drop, inplace=True)

NCES_private_schools.head()

,PSS_SCHOOL_ID,PSS_INST,LoGrade,HiGrade,PSS_ADDRESS,PSS_CITY,PSS_COUNTY_NO,PSS_COUNTY_FIPS,PSS_FIPS,PSS_ENROLL_UG,PSS_ENROLL_PK,PSS_ENROLL_T,PSS_ENROLL_TK12,PSS_COUNTY_NAME,PSS_ENROLL_elementary,PSS_ENROLL_middle_school,PSS_ENROLL_high_school
0,A9700074,91ST PSALM CHRISTIAN SCHOOL,2,17,2020 E BASELINE RD,PHOENIX,4013,13,4,0.0,17.0,172.0,155.0,MARICOPA,92.0,34.0,29.0
1,A1970230,A PLACE 4 EVERYONE LEARNING CENTER,2,11,7040 E SPRST SPG BLVD,MESA,4013,13,4,5.0,10.0,61.0,51.0,MARICOPA,43.0,3.0,0.0
2,A9100380,ACCEL,3,17,10251 N 35TH AVE,PHOENIX,4013,13,4,0.0,0.0,184.0,184.0,MARICOPA,55.0,55.0,74.0
3,A2100146,ACCEL EAST CAMPUS,3,17,1430 E BASELINE RD,TEMPE,4013,13,4,0.0,0.0,61.0,61.0,MARICOPA,15.0,16.0,30.0
4,A2100148,ACES-PEORIA,3,17,6815 W CACTUS RD,PEORIA,4013,13,4,0.0,0.0,NaN,NaN,MARICOPA,0.0,0.0,0.0


In [14]:
NCES_private_schools_df = pd.merge(NCES_private_schools, az_private_schools, left_on='PSS_SCHOOL_ID', right_on='PPIN')
NCES_private_schools_df.head()

,PSS_SCHOOL_ID,PSS_INST,LoGrade,HiGrade,PSS_ADDRESS,PSS_CITY,PSS_COUNTY_NO,PSS_COUNTY_FIPS,PSS_FIPS,PSS_ENROLL_UG,...,PSS_COUNTY_NAME,PSS_ENROLL_elementary,PSS_ENROLL_middle_school,PSS_ENROLL_high_school,PPIN,CNTY,LOCALE,LAT,LON,SCHOOLYEAR
0,A9700074,91ST PSALM CHRISTIAN SCHOOL,2,17,2020 E BASELINE RD,PHOENIX,4013,13,4,0.0,...,MARICOPA,92.0,34.0,29.0,A9700074,04013,11,33.379210,-112.037661,2019-2020
1,A1970230,A PLACE 4 EVERYONE LEARNING CENTER,2,11,7040 E SPRST SPG BLVD,MESA,4013,13,4,5.0,...,MARICOPA,43.0,3.0,0.0,A1970230,04013,11,33.380193,-111.680147,2019-2020
2,A9100380,ACCEL,3,17,10251 N 35TH AVE,PHOENIX,4013,13,4,0.0,...,MARICOPA,55.0,55.0,74.0,A9100380,04013,11,33.579730,-112.132691,2019-2020
3,A1900086,ADOBE ADVENTIST CHRISTIAN SCHOOL,3,13,9910 E ADOBE RD,MESA,4013,13,4,0.0,...,MARICOPA,9.0,8.0,0.0,A1900086,04013,21,33.430500,-111.616921,2019-2020
4,A0900114,AHWATUKEE FOOTHILLS MONTESSORI,2,3,3221 E CHANDLER BLVD,PHOENIX,4013,13,4,0.0,...,MARICOPA,9.0,0.0,0.0,A0900114,04013,11,33.303620,-112.012961,2019-2020


# Maricopa Public and Private School map


In [ ]:
print(maricopa_tracts.crs)
print(maricopa_block_groups.crs)

In [15]:
school_districts = school_districts.to_crs(epsg=4326)  
maricopa_block_groups = maricopa_block_groups.to_crs(epsg=4326)  

In [16]:
geometry = [Point(xy) for xy in zip(NCES_public_schools_df['LON'], NCES_public_schools_df['LAT'])]
gdf_az_public_schools = gpd.GeoDataFrame(NCES_public_schools_df, geometry=geometry)
gdf_az_public_schools.set_crs(epsg=4326, inplace=True)

gdf_az_public_schools

,NCES School ID,Low Grade*,High Grade*,School Name,District,Locale Code*,Locale*,Title I School*,Title 1 School Wide*,Students*,Teachers*,Student Teacher Ratio*,Type,Status,school type,LAT,LON,SCHOOLYEAR,geometry
0,40010601892,09,12,AAEC - Paradise Valley,Arizona Agribusiness & Equine Center Inc. (4331),11,City: Large,Yes,No,322.0,–,†,Regular,Open,High School,33.654305,-112.001380,2019-2020,POINT (-112.00138 33.65431)
1,40041701588,09,12,AAEC - SMCC Campus,Arizona Agribusiness & Equine Center Inc. (85816),11,City: Large,Yes,Yes,499.0,–,†,Regular,Open,High School,33.381603,-112.033968,2019-2020,POINT (-112.03397 33.38160)
2,40906000652,PK,08,Abraham Lincoln Traditional School,Washington Elementary School District (4260),11,City: Large,No,†,530.0,34.30000,15.4500000,Regular,Open,Elementary/Middle School,33.581300,-112.144000,2019-2020,POINT (-112.14400 33.58130)
3,40906000915,KG,06,Acacia Elementary School,Washington Elementary School District (4260),11,City: Large,Yes,Yes,763.0,44.50000,17.1500000,Regular,Open,Elementary/Middle School,33.616200,-112.123800,2019-2020,POINT (-112.12380 33.61620)
4,40089103676,KG,08,Academy of Math and Science Avondale,Academy of Mathematics and Science South Inc. ...,11,City: Large,Yes,Yes,1079.0,–,†,Regular,Open,Elementary/Middle School,33.434499,-112.286551,2019-2020,POINT (-112.28655 33.43450)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,40831000789,PK,05,Wood School,Tempe School District (4258),12,City: Midsize,Yes,Yes,499.0,43.23000,11.5400000,Regular,Open,Elementary School,33.371900,-111.951300,2019-2020,POINT (-111.95130 33.37190)
1193,40757000666,PK,06,Yavapai Elementary School,Scottsdale Unified District (4240),12,City: Midsize,Yes,Yes,176.0,12.66000,13.9000000,Regular,Open,Elementary/Middle School,33.455500,-111.917100,2019-2020,POINT (-111.91710 33.45550)
1194,40141002786,09,12,Youngker High School,Buckeye Union High School District (4284),41,Rural: Fringe,Yes,Yes,1987.0,87.50000,22.7100000,Regular,Open,High School,33.418500,-112.574800,2019-2020,POINT (-112.57480 33.41850)
1195,40497002280,PK,06,Zaharis Elementary,Mesa Unified District (4235),11,City: Large,No,†,786.0,45.02000,17.4600000,Regular,Open,Elementary/Middle School,33.455000,-111.628000,2019-2020,POINT (-111.62800 33.45500)


In [ ]:
gdf_az_public_schools = gdf_az_public_schools.to_crs(epsg=4326)  

In [17]:
random_district_index = random.choice(school_districts.index)
random_district = school_districts.loc[[random_district_index]]

crs_to_use = 'EPSG:4269'  # Example CRS, choose as per your requirement
maricopa_tracts = maricopa_tracts.to_crs(crs_to_use)
school_districts = school_districts.to_crs(crs_to_use)
random_district = random_district.to_crs(crs_to_use)
maricopa_block_groups = maricopa_block_groups.to_crs(crs_to_use)
gdf_az_public_schools = gdf_az_public_schools.to_crs(crs_to_use)

# Perform overlays
intersection_area_tract = gpd.overlay(maricopa_tracts, school_districts, how='intersection')
intersection_area_cbg = gpd.overlay(maricopa_block_groups, school_districts, how='intersection')

# Perform spatial joins with the predicate parameter
schools_in_intersection = gpd.sjoin(gdf_az_public_schools, intersection_area_tract, how='inner', predicate='intersects')
schools_in_intersection_cbg = gpd.sjoin(gdf_az_public_schools, intersection_area_cbg, how='inner', predicate='intersects')

schools_in_intersection['Students*'] = pd.to_numeric(schools_in_intersection['Students*'], errors='coerce')
first_elem_dist_value = schools_in_intersection['ElemDist'].iloc[0]

# Public School Voronoi diagram


In [18]:
num_census_blocks = len(census_df['label'])
num_districts = len(school_districts)

nearest_districts = np.zeros((num_census_blocks, num_districts))
districts_names = school_districts['ElemDist'].tolist()
districts_ids = range(1, num_districts)

for i in range(num_census_blocks):
    for j in range(num_districts):
        nearest_districts[i, j] = haversine(census_df['centroid'][i][0], census_df['centroid'][i][1],
                                           school_districts['coords'][j][0], school_districts['coords'][j][1])

np.savetxt(f"{dir}/nearest_districts.csv", nearest_districts, delimiter=",")

nearest_School_df = pd.DataFrame(nearest_districts)
nearest_School_df = pd.DataFrame(nearest_districts, columns=districts_names)

nearest_School = nearest_School_df.idxmin(axis=1)

closest_districts_df = pd.DataFrame(nearest_School, columns=["Closest_districts"])
closest_districts_df["ID"] = [districts_ids[districts_names.index(districts)] for districts in nearest_School]
closest_districts_df["ID"] = closest_districts_df["ID"].astype(int)

In [19]:
color_mapping = dict(zip(school_districts['ElemDist'], school_districts['color']))

closest_districts_df['color'] = closest_districts_df['Closest_districts'].map(color_mapping)
closest_districts_df

,Closest_districts,ID,color
0,CAVE CREEK #93,29,"(0.19215686274509805, 0.9333333333333333, 0.93..."
1,CAVE CREEK #93,29,"(0.19215686274509805, 0.9333333333333333, 0.93..."
2,FOUNTAIN HILLS #98,50,"(0.3764705882352941, 0.49411764705882355, 0.51..."
3,FOUNTAIN HILLS #98,50,"(0.3764705882352941, 0.49411764705882355, 0.51..."
4,CAVE CREEK #93,29,"(0.19215686274509805, 0.9333333333333333, 0.93..."
...,...,...,...
2500,GILA BEND #24,15,"(0.17647058823529413, 0.08627450980392157, 0.0..."
2501,KYRENE #28,4,"(0.7098039215686275, 0.1411764705882353, 0.752..."
2502,CHANDLER #80,12,"(0.17647058823529413, 0.4392156862745098, 0.85..."
2503,QUEEN CREEK #95,13,"(0.1411764705882353, 0.7411764705882353, 0.341..."


In [20]:
data_df = [census_df,closest_districts_df["ID"], closest_districts_df['color']] 

data_df = pd.concat(data_df, axis =1)
merged_df = pd.merge(maricopa_block_groups, data_df, left_on='GEOID', right_on='label', how='inner')

Voronoi_gpd = gpd.GeoDataFrame(merged_df)

In [21]:
Voronoi_gpd['color']

0       (0.592156862745098, 0.0196078431372549, 0.7333...
1       (0.9921568627450981, 0.2901960784313726, 0.450...
2       (0.5764705882352941, 0.23137254901960785, 0.63...
3       (0.5764705882352941, 0.23137254901960785, 0.63...
4       (0.5764705882352941, 0.23137254901960785, 0.63...
                              ...                        
2500    (0.17647058823529413, 0.4392156862745098, 0.85...
2501    (0.17647058823529413, 0.4392156862745098, 0.85...
2502    (0.17647058823529413, 0.4392156862745098, 0.85...
2503    (0.17647058823529413, 0.4392156862745098, 0.85...
2504    (0.17647058823529413, 0.4392156862745098, 0.85...
Name: color, Length: 2505, dtype: object

In [22]:
Voronoi_dissolved_gdf = Voronoi_gpd.dissolve(by='ID', 
                                             aggfunc={
                                                 "population": "sum",
                                                 "population_by_age": lambda x: np.sum(np.array(x)),
                                                 "tract_preschool_enrollment": lambda x: list(x),
                                                 'color': 'first'
                                             }).reset_index()
Voronoi_dissolved_gdf

,ID,geometry,population,population_by_age,tract_preschool_enrollment,color
0,1,"POLYGON ((-112.25504 33.54304, -112.25503 33.5...",236251,"[16655, 16669, 16998, 16339, 134300, 35290]","[1096, 1184, 1597, 1319, 888, 864, 372, 214, 1...","(0.5764705882352941, 0.23137254901960785, 0.63..."
1,2,"POLYGON ((-112.47844 33.35508, -112.47835 33.3...",57363,"[4390, 4345, 5013, 4664, 33179, 5772]","[1595, 3593, 1558, 1558, 2592, 2592, 464, 2592...","(0.5019607843137255, 0.7372549019607844, 0.913..."
2,4,"POLYGON ((-112.14005 33.29044, -112.13905 33.2...",119661,"[6700, 7312, 8340, 7692, 75745, 13872]","[902, 908, 1964, 1353, 1084, 1515, 474, 885, 6...","(0.7098039215686275, 0.1411764705882353, 0.752..."
3,5,"POLYGON ((-111.79563 33.40418, -111.79580 33.4...",316713,"[18721, 19004, 20191, 19276, 168803, 70718]","[1346, 1439, 643, 389, 250, 673, 840, 1591, 67...","(0.592156862745098, 0.0196078431372549, 0.7333..."
4,6,"POLYGON ((-111.95446 33.34916, -111.95760 33.3...",300968,"[16901, 15031, 14829, 24699, 198800, 30708]","[1207, 3646, 1616, 927, 3646, 1798, 1993, 2143...","(0.9529411764705882, 0.7843137254901961, 0.403..."
5,7,"POLYGON ((-112.60807 33.36848, -112.60807 33.3...",55732,"[4184, 5322, 5232, 4480, 30462, 6052]","[3306, 2504, 1333, 1935, 1333, 3900, 3900, 188...","(0.24705882352941178, 0.7529411764705882, 0.12..."
6,8,"POLYGON ((-112.75798 33.45722, -112.75617 33.4...",15288,"[956, 1552, 1333, 1333, 8784, 1330]","[995, 995, 1884, 2480, 2480, 2480, 2480, 2480]","(0.2901960784313726, 0.5058823529411764, 0.854..."
7,9,"POLYGON ((-111.70750 33.20464, -111.70779 33.2...",129992,"[10413, 10807, 12238, 10263, 71990, 14281]","[1019, 1019, 662, 1038, 1366, 2366, 2952, 2952...","(0.4823529411764706, 0.5019607843137255, 0.023..."
8,10,"POLYGON ((-111.82578 33.31346, -111.82632 33.3...",246350,"[17294, 17775, 19260, 17614, 151173, 23234]","[822, 1346, 1949, 597, 812, 1499, 732, 1155, 1...","(0.9647058823529412, 0.1450980392156863, 0.227..."
9,11,"POLYGON ((-113.33507 33.70464, -113.33507 33.7...",2463,"[54, 98, 100, 84, 1153, 974]",[690],"(0.7411764705882353, 0.48627450980392156, 0.11..."


# Precentage Base Diagram


In [23]:
projected_crs = 'EPSG:26912'

percentage_results = []

for idx, tract in maricopa_block_groups.iterrows():
    tract_gdf = gpd.GeoDataFrame([tract], geometry='geometry', crs=maricopa_block_groups.crs)
    tract_gdf_projected = tract_gdf.to_crs(projected_crs)
    intersection_result = gpd.overlay(tract_gdf_projected, school_districts.to_crs(projected_crs), how='intersection')

    for idx, district in intersection_result.iterrows():
        intersection_area = district.geometry.area
        total_tract_area = tract_gdf_projected.geometry.area.iloc[0] if tract_gdf_projected.geometry.area.iloc[0] > 0 else 1.0
        percentage_within_district = (intersection_area / total_tract_area) * 100
        
        percentage_results.append({
            'GEOID': tract['GEOID'],  
            'ElemDist': district['ElemDist'],  
            'PercentageWithinDistrict': percentage_within_district,
        })

In [24]:
precentage_schools = schools_in_intersection_cbg[['NCES School ID', 'GEOID','ElemDist', 'School Name', 'Students*', 'school type']]
precentage_schools

,NCES School ID,GEOID,ElemDist,School Name,Students*,school type
0,40010601892,040136196001,PARADISE VALLEY #69,AAEC - Paradise Valley,322.0,High School
1,40041701588,040131163003,ROOSEVELT #66,AAEC - SMCC Campus,499.0,High School
2,40906000652,040131042051,WASHINGTON #6,Abraham Lincoln Traditional School,530.0,Elementary/Middle School
3,40906000915,040131041002,WASHINGTON #6,Acacia Elementary School,763.0,Elementary/Middle School
4,40089103676,040130822031,LITTLETON #65,Academy of Math and Science Avondale,1079.0,Elementary/Middle School
...,...,...,...,...,...,...
1192,40831000789,040133200074,TEMPE #3,Wood School,499.0,Elementary School
1193,40757000666,040132182002,SCOTTSDALE #48,Yavapai Elementary School,176.0,Elementary/Middle School
1194,40141002786,040130506082,BUCKEYE #33,Youngker High School,1987.0,High School
1195,40497002280,040134201102,MESA #4,Zaharis Elementary,786.0,Elementary/Middle School


In [25]:
color_mapping = dict(zip(school_districts['ElemDist'], school_districts['color']))

closest_districts_df['color'] = closest_districts_df['Closest_districts'].map(color_mapping)
closest_districts_df

,Closest_districts,ID,color
0,CAVE CREEK #93,29,"(0.19215686274509805, 0.9333333333333333, 0.93..."
1,CAVE CREEK #93,29,"(0.19215686274509805, 0.9333333333333333, 0.93..."
2,FOUNTAIN HILLS #98,50,"(0.3764705882352941, 0.49411764705882355, 0.51..."
3,FOUNTAIN HILLS #98,50,"(0.3764705882352941, 0.49411764705882355, 0.51..."
4,CAVE CREEK #93,29,"(0.19215686274509805, 0.9333333333333333, 0.93..."
...,...,...,...
2500,GILA BEND #24,15,"(0.17647058823529413, 0.08627450980392157, 0.0..."
2501,KYRENE #28,4,"(0.7098039215686275, 0.1411764705882353, 0.752..."
2502,CHANDLER #80,12,"(0.17647058823529413, 0.4392156862745098, 0.85..."
2503,QUEEN CREEK #95,13,"(0.1411764705882353, 0.7411764705882353, 0.341..."


In [26]:
percentage_results_df = pd.DataFrame(percentage_results)

pivoted_df = percentage_results_df.pivot_table(index='GEOID', columns='ElemDist', values='PercentageWithinDistrict', aggfunc='mean')
pivoted_df = pivoted_df.fillna(0)
pivoted_df = pivoted_df.reset_index()
pivoted_df.iloc[:, 1:] = pivoted_df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
pivoted_df['Tract School District'] = pivoted_df.iloc[:, 1:].idxmax(axis=1)

color_mapping = dict(zip(school_districts['ElemDist'], school_districts['color']))
pivoted_df['precent color'] = pivoted_df['Tract School District'].map(color_mapping)


data_df_1 = pivoted_df[["GEOID", "Tract School District", "precent color"]]

merged_df = pd.merge(left=pd.merge(maricopa_block_groups, data_df_1, on='GEOID', how='inner'),
                     right=census_df,
                     left_on='GEOID', 
                     right_on='label', 
                     how='inner')

# Merge with precentage_schools
merged_df = pd.merge(merged_df, precentage_schools, on='GEOID', how='left')

# Fill NaN values with 0 and drop duplicates based on 'GEOID'
merged_df.fillna(0, inplace=True)
merged_df.drop_duplicates(subset=['GEOID'], inplace=True)

percentage_gpd_df = gpd.GeoDataFrame(merged_df).reset_index()
percentage_gpd_df

,index,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,median_income,average_household_size,pop_density_km2,tract_median_income,tract_gini_index,NCES School ID,ElemDist,School Name,Students*,school type
0,0,04,013,421702,3,040134217023,Block Group 3,G5030,S,310375,...,30610,2.31,3824.405961,32552,0.4616,0.000000e+00,0,0,0.0,0
1,1,04,013,040531,2,040130405312,Block Group 2,G5030,S,616061,...,56500,1.67,1173.585083,45236,0.4099,0.000000e+00,0,0,0.0,0
2,2,04,013,092312,1,040130923121,Block Group 1,G5030,S,295623,...,60028,3.33,3734.486153,34271,0.4760,0.000000e+00,0,0,0.0,0
3,3,04,013,092721,1,040130927211,Block Group 1,G5030,S,1108648,...,101542,3.44,2002.439007,106471,0.3012,0.000000e+00,0,0,0.0,0
4,4,04,013,093104,1,040130931041,Block Group 1,G5030,S,168697,...,24645,2.77,11742.947415,31321,0.3800,0.000000e+00,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2471,2891,04,013,811900,2,040138119002,Block Group 2,G5030,S,771839,...,161042,3.02,1680.402260,108603,0.3877,0.000000e+00,0,0,0.0,0
2472,2892,04,013,812000,1,040138120001,Block Group 1,G5030,S,1279036,...,70101,4.13,2307.206365,70071,0.3895,0.000000e+00,0,0,0.0,0
2473,2893,04,013,812000,2,040138120002,Block Group 2,G5030,S,1360898,...,70000,2.10,1497.540595,70071,0.3895,0.000000e+00,0,0,0.0,0
2474,2894,04,013,523002,3,040135230023,Block Group 3,G5030,S,666349,...,64091,2.77,1992.949641,41989,0.4526,0.000000e+00,0,0,0.0,0


In [27]:
num_census_blocks = len(percentage_gpd_df['label'])
num_schools = len(schools_in_intersection_cbg)

nearest_school = np.zeros((num_census_blocks, num_schools))
School_names = schools_in_intersection_cbg['School Name'].tolist()
school_ids = range(1, num_schools)

In [28]:
schools_in_intersection_cbg.sort_values(by='ElemDist', inplace=True)

current_district = None
adjustment_factor = 1.0
increment = 0.05 

schools_in_intersection_cbg['school_color'] = None

for idx, row in schools_in_intersection_cbg.iterrows():
    if row['ElemDist'] != current_district:
        current_district = row['ElemDist']
        adjustment_factor = 1.0
    else:
        adjustment_factor += increment
    
    schools_in_intersection_cbg.at[idx, 'school_color'] = adjust_lightness(row['color'], amount=adjustment_factor)

schools_in_intersection_cbg[['ElemDist', 'School Name', 'color', 'school_color']]

,ElemDist,School Name,color,school_color
18,AGUILA #63,Aguila Pre-School,"(0.7411764705882353, 0.48627450980392156, 0.11...","(0.7411764705882353, 0.48627450980392156, 0.11..."
17,AGUILA #63,Aguila Elementary School,"(0.7411764705882353, 0.48627450980392156, 0.11...","(0.7782352941176471, 0.5105882352941177, 0.123..."
694,ALHAMBRA #68,Madrid Neighborhood School,"(0.5294117647058824, 0.6509803921568628, 0.874...","(0.5294117647058824, 0.6509803921568628, 0.874..."
212,ALHAMBRA #68,Choice Learning Academy,"(0.5294117647058824, 0.6509803921568628, 0.874...","(0.5558823529411765, 0.6835294117647059, 0.918..."
982,ALHAMBRA #68,Southwest Key Transitional Learning Center,"(0.5294117647058824, 0.6509803921568628, 0.874...","(0.5823529411764706, 0.7160784313725491, 0.961..."
...,...,...,...,...
391,WICKENBURG #9,Festival Foothills Elementary School,"(0.7294117647058823, 0.611764705882353, 0.3294...","(0.911764705882353, 0.7647058823529413, 0.4117..."
783,WILSON #7,Nueva Vista School / Arizona State Hospital,"(0.43529411764705883, 0.7098039215686275, 0.61...","(0.43529411764705883, 0.7098039215686275, 0.61..."
1189,WILSON #7,Wilson Elementary School,"(0.43529411764705883, 0.7098039215686275, 0.61...","(0.4570588235294118, 0.7452941176470589, 0.642..."
1190,WILSON #7,Wilson Primary School,"(0.43529411764705883, 0.7098039215686275, 0.61...","(0.47882352941176476, 0.7807843137254903, 0.67..."


In [29]:
num_census_blocks = len(percentage_gpd_df['label'])
num_schools = len(schools_in_intersection_cbg)

nearest_school = np.zeros((num_census_blocks, num_schools))
School_names = schools_in_intersection_cbg['School Name'].tolist()
school_ids = range(1, num_schools)

In [1]:
preschools = schools_in_intersection_cbg[
    schools_in_intersection_cbg['school type'].isin(['Preschool'])]
preschools_names = preschools['School Name'].tolist()

elementary_schools = schools_in_intersection_cbg[
    schools_in_intersection_cbg['school type'].isin(['Elementary School', 'Elementary/Middle School', 'Elementary/Middle/High School'])]
elementary_schools_names = elementary_schools['School Name'].tolist()

middle_schools = schools_in_intersection_cbg[
    schools_in_intersection_cbg['school type'].isin(['Middle School', 'Elementary/Middle School', 'Middle/High School', 'Elementary/Middle/High School'])]
middle_schools_names = middle_schools['School Name'].tolist()

high_schools = schools_in_intersection_cbg[
    schools_in_intersection_cbg['school type'].isin(['High School', 'Middle/High School', 'Elementary/Middle/High School'])]
high_schools_names = high_schools['School Name'].tolist()

NameError: name 'schools_in_intersection_cbg' is not defined

In [31]:
num_preschools = len(preschools)
num_elementary_schools = len(elementary_schools)
num_middle_schools = len(middle_schools)
num_high_schools = len(high_schools)

nearest_preschools = np.zeros((num_census_blocks, num_preschools))
nearest_elementary_schools = np.zeros((num_census_blocks, num_elementary_schools))
nearest_middle_schools = np.zeros((num_census_blocks, num_middle_schools))
nearest_high_schools= np.zeros((num_census_blocks, num_high_schools))

students_preschools = np.zeros((num_census_blocks, num_preschools))
students_elementary_schools = np.zeros((num_census_blocks, num_elementary_schools))
students_middle_schools = np.zeros((num_census_blocks, num_middle_schools))
students_high_schools= np.zeros((num_census_blocks, num_high_schools))

In [32]:
labels = census_df['label'].tolist()

matrix_df = pd.DataFrame(0, index=labels, columns=labels)
matrix_df

,040130101011,040130101012,040130101013,040130101021,040130101022,040130101023,040130304011,040130304012,040130304013,040130304021,...,040139412001,040139413001,040139413002,040139413003,040139413004,040139801001,040139804001,040139805001,040139806001,040139807001
040130101011,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
040130101012,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
040130101013,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
040130101021,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
040130101022,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
040139801001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
040139804001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
040139805001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
040139806001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
for i in range(num_census_blocks):
    tract_district = percentage_gpd_df['Tract School District'][i]
    student_home_cbg = percentage_gpd_df['GEOID'][i]
    students_population_cbg = percentage_gpd_df['population_by_age'][i][:3]
    
    for j in range(num_preschools):
        school_district = preschools['ElemDist'].iloc[j]
        school_district_cbg = preschools['GEOID'].iloc[j]
        school_district_population = preschools['Students*'].iloc[j]

        if tract_district == school_district:
            try:
                nearest_preschools[i, j] = haversine(percentage_gpd_df['centroid'][i][0], 
                                                            percentage_gpd_df['centroid'][i][1],
                                                            preschools['coords'].iloc[j][0], 
                                                            preschools['coords'].iloc[j][1])
                matrix_df.at[student_home_cbg, school_district_cbg] = students_population_cbg[0]

            except KeyError:
                print(f"KeyError encountered at index: {j}")
        else:
            nearest_elementary_schools[i, j] = None

    for j in range(num_elementary_schools):
        school_district = elementary_schools['ElemDist'].iloc[j]
        school_district_cbg = elementary_schools['GEOID'].iloc[j]
        school_district_population = elementary_schools['Students*'].iloc[j]

        if tract_district == school_district:
            try:
                nearest_elementary_schools[i, j] = haversine(percentage_gpd_df['centroid'][i][0], 
                                                            percentage_gpd_df['centroid'][i][1],
                                                            elementary_schools['coords'].iloc[j][0], 
                                                            elementary_schools['coords'].iloc[j][1])
                matrix_df.at[student_home_cbg, school_district_cbg] = students_population_cbg[0]

            except KeyError:
                print(f"KeyError encountered at index: {j}")
        else:
            nearest_elementary_schools[i, j] = None
    for j in range(num_middle_schools):
        school_district = middle_schools['ElemDist'].iloc[j]
        school_district_cbg = middle_schools['GEOID'].iloc[j]
        school_district_population = middle_schools['Students*'].iloc[j]

        if tract_district == school_district:
            try:
                nearest_middle_schools[i, j] = haversine(percentage_gpd_df['centroid'][i][0], 
                                                            percentage_gpd_df['centroid'][i][1],
                                                            middle_schools['coords'].iloc[j][0], 
                                                            middle_schools['coords'].iloc[j][1])
                matrix_df.at[student_home_cbg, school_district_cbg] = students_population_cbg[1].sum()
            except KeyError:
                print(f"KeyError encountered at index: {j}")
        else:
            nearest_middle_schools[i, j] = None

    for j in range(num_high_schools):
        school_district = high_schools['ElemDist'].iloc[j]
        school_district_cbg = high_schools['GEOID'].iloc[j]
        school_district_population = high_schools['Students*'].iloc[j]

        if tract_district == school_district:
            try:
                nearest_middle_schools[i, j] = haversine(percentage_gpd_df['centroid'][i][0], 
                                                            percentage_gpd_df['centroid'][i][1],
                                                            high_schools['coords'].iloc[j][0], 
                                                            high_schools['coords'].iloc[j][1])
                matrix_df.at[student_home_cbg, school_district_cbg] = students_population_cbg[1].sum()
            except KeyError:
                print(f"KeyError encountered at index: {j}")
        else:
            nearest_high_schools[i, j] = None

In [ ]:
closest_elementary_df = pd.DataFrame(nearest_elementary, columns=["Elementary School Name"])

In [34]:
nearest_elementary_school_df = pd.DataFrame(nearest_elementary_schools, columns = elementary_schools_names)
nearest_middle_school_df = pd.DataFrame(nearest_middle_schools, columns = middle_schools_names)
nearest_high_schools_df = pd.DataFrame(nearest_high_schools, columns = high_schools_names)

nearest_elementary = nearest_elementary_school_df.idxmin(axis=1)
nearest_middle = nearest_middle_school_df.idxmin(axis=1)
nearest_high = nearest_high_schools_df.idxmin(axis=1)

closest_elementary_df = pd.DataFrame(nearest_elementary, columns=["Elementary School Name"])
closest_middle_df = pd.DataFrame(nearest_middle, columns=[" Middle School Name"])
closest_high_df = pd.DataFrame(nearest_high, columns=["High School Name"])

/var/folders/_v/4q2_l88n1vndy51_ksbfylnc0000gn/T/ipykernel_46999/4245327498.py:7: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  nearest_high = nearest_high_schools_df.idxmin(axis=1)


In [41]:
closest_school_df

,School Name,ID
0,CAVE CREEK #93,NaN
1,CAVE CREEK #93,NaN
2,FOUNTAIN HILLS #98,NaN
3,FOUNTAIN HILLS #98,NaN
4,CAVE CREEK #93,NaN
...,...,...
2500,GILA BEND #24,NaN
2501,KYRENE #28,NaN
2502,CHANDLER #80,NaN
2503,QUEEN CREEK #95,NaN


In [39]:
nearest_School = nearest_School_df.idxmin(axis=1)
closest_school_df = pd.DataFrame(nearest_School, columns=["School Name"])

school_name_to_id = {name: i for i, name in enumerate(School_names)}

closest_school_df["ID"] = closest_school_df["School Name"].map(school_name_to_id)
closest_school_df["ID"] = closest_school_df["ID"].astype(int)

for i in range(num_census_blocks):
    nearest_school_id = closest_school_df["ID"].iloc[i]

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [37]:
color_mapping = dict(zip(schools_in_intersection_cbg['School Name'], schools_in_intersection_cbg['school_color']))

closest_school_df['Color'] = closest_school_df['School Name'].map(color_mapping)
closest_school_df

NameError: name 'closest_school_df' is not defined

In [36]:
closest_elementary_df = pd.DataFrame(nearest_elementary, columns=["Elementary School Name"])
closest_middle_df = pd.DataFrame(nearest_middle, columns=[" Middle School Name"])
closest_high_df = pd.DataFrame(nearest_high, columns=[" High School Name"])


In [42]:
data_df = pd.concat([percentage_gpd_df, closest_school_df["School Name"], closest_elementary_df, closest_middle_df, closest_high_df ], axis=1)

percentage_Voronoi_gpd = gpd.GeoDataFrame(data_df)
percentage_Voronoi_gpd

,index,STATEFP,COUNTYFP,TRACTCE,BLKGRPCE,GEOID,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,tract_gini_index,NCES School ID,ElemDist,School Name,Students*,school type,School Name,Elementary School Name,Middle School Name,High School Name
0,0.0,04,013,421702,3,040134217023,Block Group 3,G5030,S,310375.0,...,0.4616,0.0,0,0,0.0,0,CAVE CREEK #93,MacArthur Elementary School,Constitution Elementary School,Arizona Agribusiness & Equine Center Inc. - Mesa
1,1.0,04,013,040531,2,040130405312,Block Group 2,G5030,S,616061.0,...,0.4099,0.0,0,0,0.0,0,CAVE CREEK #93,Freedom Traditional Academy,Sanborn Elementary School,Dysart High School
2,2.0,04,013,092312,1,040130923121,Block Group 1,G5030,S,295623.0,...,0.4760,0.0,0,0,0.0,0,FOUNTAIN HILLS #98,Glendale American School,Willis Junior High School,Glendale Union High School District Online Lea...
3,3.0,04,013,092721,1,040130927211,Block Group 1,G5030,S,1108648.0,...,0.3012,0.0,0,0,0.0,0,FOUNTAIN HILLS #98,Desert Valley Elementary School,Greenfield Junior High School,RCB College Preparatory Academy
4,4.0,04,013,093104,1,040130931041,Block Group 1,G5030,S,168697.0,...,0.3800,0.0,0,0,0.0,0,CAVE CREEK #93,Madrid Neighborhood School,Aguila Elementary School,Southwest Key Transitional Learning Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,GILA BEND #24,NaN,NaN,NaN
2501,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,KYRENE #28,NaN,NaN,NaN
2502,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,CHANDLER #80,NaN,NaN,NaN
2503,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,QUEEN CREEK #95,NaN,NaN,NaN


In [44]:
dir

'/Users/alexandernez/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/Movement_model'

In [45]:
file_path = '/Users/alexandernez/Desktop/Github/EpiMoRPH-Modeling/social-determinate-of-health/scratch/Movement_model/voroni_school.csv'
percentage_Voronoi_gpd.to_csv(file_path)

In [ ]:
percentage_Voronoi_gpd['Preschools Name'].unique()

In [ ]:
census_df['population'] = pd.to_numeric(census_df['population'], errors='coerce')

census_tract_df = census_df.groupby('tract_geoid').agg({
    'population': 'sum',
    'population_by_age':lambda x: np.sum(np.array(x)),
    'tract_preschool_enrollment': 'first', 
    'tract_kindergarten_enrollment': 'first', 
    'tract_elementary_enrollment': 'first', 
    'tract_middle_school_enrollment': 'first', 
    'tract_high_school_enrollment': 'first', 
}).reset_index()
census_tract_df

In [ ]:

census_tract_df = percentage_gpd_df.groupby('tract_geoid').agg({
    'population': 'sum',
    'population_by_age':lambda x: np.sum(np.array(x)),
    'tract_preschool_enrollment': 'first', 
    'tract_kindergarten_enrollment': 'first', 
    'tract_elementary_enrollment': 'first', 
    'tract_middle_school_enrollment': 'first', 
    'tract_high_school_enrollment': 'first', 
}).reset_index()
census_tract_df

In [ ]:
cgb_district = percentage_gpd_df[['GEOID', 'Tract School District']]

test_df = pd.merge(merged_df, cgb_district, left_on='label', right_on='GEOID', how='left')

test_df['Tract School District'].fillna('none', inplace=True)

test_df = test_df.drop_duplicates(subset='label')

test_df

In [ ]:
Voronoi_dissolved_gdf
percentage_Voronoi_gpd

In [ ]:
percentage_dissolved_gdf = percentage_gpd_df.dissolve(by='Tract School District').reset_index()
percentage_dissolved_gdf

In [ ]:
gdf_az_public_schools.crs = 'EPSG:4269'

intersection = gpd.overlay(gdf_az_public_schools, maricopa_block_groups, how='intersection')

intersection

# Plotting NCES


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

closest_districts_df['Closest_districts'].value_counts().plot(kind='bar')
plt.title('Distribution of Closest School Districts for Census Blocks')
plt.xlabel('School District')
plt.ylabel('Number of Census Blocks')
plt.show()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
centroids_gdf = gpd.GeoDataFrame(geometry=school_districts['Centroid'])

fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

maricopa_main_roads.plot(ax=ax, color='blue', linewidth=0.2)
maricopa_secondary_roads.plot(ax=ax, color='blue', linewidth=0.2)

school_districts.plot(linewidth=0.05, ax=ax, edgecolor='black', color=random_colors, alpha=0.5)
centroids_gdf.plot(marker='o', color='red', markersize=1, ax=ax)
maricopa_block_groups.plot(linewidth=0.2, ax=ax, edgecolor='black', color='none', alpha=0.6)

ax.axis('off')
plt.title('Maricopa School Districts with Centroids')
plt.show()

In [ ]:
def plot_maricopa_map(maricopa_main_roads, maricopa_tracts, school_districts, random_colors):
    fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

    maricopa_main_roads.plot(ax=ax, color='blue', linewidth=0.2)

    maricopa_tracts.plot(linewidth=0.1, ax=ax, edgecolor='black', color='none')
    school_districts.plot(linewidth=0.05, ax=ax, edgecolor='black', color=random_colors, alpha=0.4)

    ax.axis('off')

    plt.title('Maricopa School Districts')
    plt.show()


plot_maricopa_map(maricopa_roads, maricopa_tracts, school_districts, random_colors)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

maricopa_main_roads.plot(ax=ax, color='blue', linewidth=0.2)
maricopa_secondary_roads.plot(ax=ax, color='blue', linewidth=0.2)

maricopa_tracts.plot(linewidth=0.1, ax=ax, edgecolor='black', color = 'none')
school_districts.plot(linewidth=0.05, ax=ax, edgecolor='black', color = random_colors, alpha = 0.4)

ax.axis('off')

plt.title('Maricopa School Districts')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)


percentage_Voronoi_gpd['Color'].fillna('grey', inplace=True)

percentage_Voronoi_gpd.plot(ax=ax, marker='o', color=percentage_Voronoi_gpd['Color'], markersize=0.5, linewidth=0.5)

percentage_dissolved_gdf.plot(linewidth=0.5, ax=ax, edgecolor='black', color='none')

maricopa_block_groups.plot(linewidth=0.2, ax=ax, edgecolor='black', color='none', alpha=0.6)
maricopa_main_roads.plot(ax=ax, color='blue', linewidth=0.2)

ax.axis('off')
plt.title('Maricopa School Districts (Voronoi)')
plt.show()


In [ ]:
minx, miny, maxx, maxy = [-111.9, 33.25, -111.5, 33.4]

ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
minx, miny, maxx, maxy = [-111.9, 33.28, -111.6, 33.45]

ax.set_xlim(minx, maxx)
ax.set_ylim(miny, maxy)
Voronoi_gpd.plot(ax=ax, color=Voronoi_gpd['color'], markersize=0.5, linewidth=0.5)
Voronoi_dissolved_gdf.plot(linewidth=0.5, ax=ax, edgecolor='black', color='none')


maricopa_block_groups.plot(linewidth=0.2, ax=ax, edgecolor='black', color='none')
maricopa_main_roads.plot(ax=ax, color='blue', linewidth=0.2)

ax.axis('off')

plt.title('Maricopa School Districts (Voronoi)')
plt.show()

In [ ]:
resam_colors = plt.cm.turbo.resampled(num_groups)
colors = [resam_colors(i) for i in np.linspace(0, 1, num=len(gpd_df['ID'].unique()))]
random.shuffle(colors)

In [ ]:
Voronoi_dissolved_gdf

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), dpi=300)

percentage_Voronoi_gpd.plot(ax=ax1, color=percentage_Voronoi_gpd['precent color'], markersize=0.5, linewidth=0.05)
percentage_dissolved_gdf.plot(ax=ax1, color='none', markersize=0.5, linewidth=0.5)
maricopa_block_groups.plot(ax=ax1, edgecolor='black', color='none', linewidth=0.1, label='Maricopa')
centroids_gdf.plot(marker='o', color='black', markersize=1, ax=ax1)

ax1.set_title("Districts Precentage Diagram")
ax1.axis('off')

school_districts.plot(linewidth=0.05, ax=ax2, edgecolor='black', color=school_districts['color'])
centroids_gdf.plot(marker='o', color='black', markersize=1, ax=ax2)
maricopa_block_groups.plot(linewidth=0.1, ax=ax2, edgecolor='black', color='none')

ax2.set_title('Maricopa School Districts')
ax2.axis('off')

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10), dpi=300)

percentage_dissolved_gdf.plot(linewidth=0.5, ax=ax1, edgecolor='black', color=percentage_dissolved_gdf['color'])

maricopa_block_groups.plot(ax=ax1, edgecolor='black', color='none', linewidth=0.1, label='Maricopa')
centroids_gdf.plot(marker='o', color='black', markersize=1, ax=ax1)

ax1.set_title("Districts Voronoi Diagram")
ax1.axis('off')

school_districts.plot(linewidth=0.5, ax=ax2, edgecolor='black', color=random_colors)
centroids_gdf.plot(marker='o', color='black', markersize=1, ax=ax2)
maricopa_block_groups.plot(linewidth=0.1, ax=ax2, edgecolor='black', color='none')

ax2.set_title('Maricopa School Districts')
ax2.axis('off')

plt.show()

In [ ]:
geometry = [Point(xy) for xy in zip(NCES_private_schools_df['LON'], NCES_private_schools_df['LAT'])]
gdf_private_schools = gpd.GeoDataFrame(NCES_private_schools_df, geometry=geometry)

fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
maricopa_tracts.plot(linewidth=0.2, ax=ax, edgecolor='black', color='none', alpha=0.6)
school_districts.plot(linewidth=0.05, ax=ax, edgecolor='black', color=random_colors, alpha=0.4)

gdf_private_schools.plot(ax=ax, marker='o', color='red', markersize=1, linewidth=1)

ax.axis('off')
plt.title('Maricopa School Districts with Private Schools')
plt.show()

In [ ]:
schools_in_intersection_cbg['Students*'] = pd.to_numeric(schools_in_intersection_cbg['Students*'], errors='coerce')


In [ ]:
GILBERT = marciopa_schools.groupby('GEOID').agg({
    'NCES School ID': list,
    'School Name': list,
    'District': list,
    'school type': list,
    'Students*': 'sum',
}).reset_index()
GILBERT 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

intersection_area_cbg.plot(ax=ax, linewidth=0.2, edgecolor='black', color=intersection_area_cbg['color'][0], alpha=0.5)

maricopa_tracts.plot(linewidth=0.2, ax=ax, edgecolor='black', color='none', alpha=0.6)

schools_in_intersection.plot(ax=ax, marker='o', color='red', markersize=0.75, linewidth=0.75) 

ax.axis('off')
ax.set_title(f'School District: {first_elem_dist_value}')
plt.show()


In [ ]:

    
schools_in_intersection_cbg['school type'] = schools_in_intersection_cbg.apply(lambda row: classify_school_type(row['Low Grade*'], row['High Grade*']), axis=1)
schools_in_intersection_cbg['school type']

In [ ]:
schools_in_intersection['school type'] = schools_in_intersection.apply(lambda row: classify_school_type(row['Low Grade*'], row['High Grade*']), axis=1)
schools_in_intersection['school type']

In [ ]:
percentage_Voronoi_gpd.plot()

In [ ]:
schools_in_intersection

In [ ]:
preschools = schools_in_intersection_cbg[
    schools_in_intersection_cbg['school type'].isin(['Preschool'])]
preschools_names = preschools['School Name'].tolist()

elementary_schools = schools_in_intersection_cbg[
    schools_in_intersection_cbg['school type'].isin(['Elementary School', 'Elementary/Middle School', 'Elementary/Middle/High School'])]
elementary_schools_names = elementary_schools['School Name'].tolist()

middle_schools = schools_in_intersection_cbg[
    schools_in_intersection_cbg['school type'].isin(['Middle School', 'Elementary/Middle School', 'Middle/High School', 'Elementary/Middle/High School'])]
middle_schools_names = middle_schools['School Name'].tolist()

high_schools = schools_in_intersection_cbg[
    schools_in_intersection_cbg['school type'].isin(['High School', 'Middle/High School', 'Elementary/Middle/High School'])]
high_schools_names = high_schools['School Name'].tolist()

In [ ]:
elementary_schools

In [ ]:
percentage_dissolved_gdf

In [ ]:
sum_elementary = preschools['Students*'].sum()
sum_elementary = elementary_schools['Students*'].sum()
sum_middle = middle_schools['Students*'].sum()
sum_high = high_schools['Students*'].sum()

fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

percentage_dissolved_gdf.plot(linewidth=0.5, ax=ax, edgecolor='black', color=percentage_dissolved_gdf['color'], alpha = 0.66)
maricopa_block_groups.plot(ax=ax, edgecolor='black', color='none',linewidth=0.1, alpha=0.5)

elementary_schools.plot(ax=ax, marker='o', color='yellow', markersize=0.5, linewidth=0.5)
middle_schools.plot(ax=ax, marker='o', color='green', markersize=0.5, linewidth=0.5)
high_schools.plot(ax=ax, marker='o', color='blue', markersize=0.5, linewidth=0.5)

elementary_patch = mpatches.Patch(color='yellow', label=f'Elementary (Students: {sum_elementary})')
middle_patch = mpatches.Patch(color='green', label=f'Middle (Students: {sum_middle})')
high_patch = mpatches.Patch(color='blue', label=f'High (Students: {sum_high})')


plt.legend(handles=[elementary_patch, middle_patch, high_patch], loc='lower right')

ax.axis('off')
ax.set_title(f'School District')
plt.show()

In [ ]:

fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
maricopa_tracts.plot(linewidth=0.2, ax=ax, edgecolor='black', color='none', alpha=0.6)
school_districts.plot(linewidth=0.05, ax=ax, edgecolor='black', color=random_colors, alpha=0.4)

elementary_schools.plot(ax=ax, marker='o', color='yellow', markersize=0.5, linewidth=0.5, label='Elementary')
middle_schools.plot(ax=ax, marker='o', color='green', markersize=0.5, linewidth=0.5, label='Middle')
high_schools.plot(ax=ax, marker='o', color='blue', markersize=0.5, linewidth=0.5, label='High')

elementary_patch = mpatches.Patch(color='yellow', label='Elementary')
middle_patch = mpatches.Patch(color='green', label='Middle')
high_patch = mpatches.Patch(color='blue', label='High')

plt.legend(handles=[elementary_patch, middle_patch, high_patch])

ax.axis('off')
plt.title('Maricopa School Districts with Public Schools')
plt.show()

In [ ]:
intersection_area

In [ ]:
intersection_area_cbg

In [ ]:

sum_school = schools_in_intersection['Students*'].sum()
first_elem_dist_value = schools_in_intersection['ElemDist'].iloc[0]  # Adjust this if needed

unique_names = intersection_area_cbg['NAMELSAD'].unique()

filtered_tracts = maricopa_tracts[maricopa_tracts['NAMELSAD'].isin(unique_names)]

fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

for name in unique_names:
    tract_to_plot = filtered_tracts[filtered_tracts['NAMELSAD'] == name]
    tract_to_plot.plot(linewidth=0.2, ax=ax, edgecolor='black', label=name)


if 'color' in intersection_area_cbg.columns:
    intersection_area_cbg.plot(linewidth=0.2, ax=ax, edgecolor='black', color=intersection_area['color'].iloc[0], alpha=0.6)
else:
    intersection_area_cbg.plot(linewidth=0.2, ax=ax, edgecolor='black', alpha=0.6)


school_patch = mpatches.Patch(color='yellow', label=f'{first_elem_dist_value} (Students: {sum_school})')
schools_in_intersection.plot(ax=ax, marker='o', color='yellow', markersize=0.75, linewidth=0.75) 

ax.set_title(f'School District: {first_elem_dist_value}')
ax.legend(handles=[school_patch])
ax.axis('off')

plt.show()
